In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!git clone https://github.com/X-2-M/HERO.git

Cloning into 'HERO'...
remote: Enumerating objects: 3988, done.
remote: Counting objects: 100% (3988/3988), done.
remote: Compressing objects: 100% (2195/2195), done.
remote: Total 3988 (delta 1806), reused 3952 (delta 1788), pack-reused 0
Receiving objects: 100% (3988/3988), 18.21 MiB | 6.69 MiB/s, done.
Resolving deltas: 100% (1806/1806), done.


In [ ]:
# 检查当前路径和库状态
!pwd
!git status

/content/HERO
On branch main
Your branch is ahead of 'origin/main' by 1 commit.
  (use "git push" to publish your local commits)

nothing to commit, working tree clean


# 在Colab中上传一个文件到HERO对应的GitHub库

In [13]:
# 复制文件到HERO库中
!cp /content/drive/MyDrive/glove.6B.100d.txt /content/HERO/

# 进入HERO库目录
%cd /content/HERO

# 设置Git配置
!git config --global user.name "X-2-M"
!git config --global user.email "2021290207@email.szu.edu.cn"

# 使用个人访问令牌进行身份验证
import os
os.environ['GIT_ASKPASS'] = 'echo'
os.environ['GIT_USERNAME'] = 'X-2-M'
os.environ['GIT_PASSWORD'] = 'Personal Access Token'

!git lfs install

# 将文件添加到GitHub库
!git lfs track "glove.6B.100d.txt"
!git add .gitattributes glove.6B.100d.txt

# 提交更改
!git commit -m "Add glove.6B.100d.txt using Git LFS"

# 推送更改到远程GitHub库
!git push https://X-2-M:Personal Access Token@github.com/X-2-M/HERO.git



/content/HERO
Updated Git hooks.
Git LFS initialized.
"glove.6B.100d.txt" already supported
On branch main
nothing to commit, working tree clean
batch response: @X-2-M can not upload new objects to public fork X-2-M/HERO
error: failed to push some refs to 'https://github.com/X-2-M/HERO.git'


# Github库的修改更新到本地HERO库

In [63]:
# 确认当前目录
%cd /content/HERO

# 移除已经存在的远程仓库
!git remote remove origin
!git remote add origin https://github.com/X-2-M/HERO.git

# 设置默认合并策略
# !git config pull.rebase false

# 获取最新的远程仓库更新
!git fetch origin

# 查看当前的分支和状态
!git status
!git branch

# 将远程分支合并到本地分支
!git pull origin main

/content/HERO
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 6 (delta 4), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (6/6), 1.90 KiB | 1.90 MiB/s, done.
From https://github.com/X-2-M/HERO
 * [new branch]      main       -> origin/main
On branch main
All conflicts fixed but you are still merging.
  (use "git commit" to conclude merge)

Changes to be committed:
	modified:   util.py

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   data/GossipCop/story_based_fake_test.json
	modified:   data/GossipCop/story_based_fake_train.json
	modified:   data/GossipCop/story_based_fake_val.json

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	data/GossipCop/test.json
	data/GossipCop/train.json
	data/GossipCop/val.json

* main
error: 

# 数据预处理、数据划分

In [ ]:
# 格式转换
import json
import re

# 读取原始数据集
with open('/content/drive/MyDrive/gossipcop_v3_origin.json', 'r') as f:
    data = json.load(f)

# 创建新的数据结构并重新编号
formatted_data = []
new_id = 1

# for key, value in data.items():
for key, value in data.items():
    news_item = {
        "news_id": new_id,
        "title": value["title"],
        "body_text": value["text"],
          "reliability": 1 if value["label"] == "real" else 0
    }
    formatted_data.append(news_item)
    new_id += 1

    # 限定最大ID为2029
    if new_id >= 2029:
        break

# 将新数据保存为JSON文件
with open('/content/drive/MyDrive/formatted_news.json', 'w') as f:
    json.dump(formatted_data, f)


In [68]:
# 格式转换
import json
import re

# 读取原始数据集
with open('/content/drive/MyDrive/gossipcop_v3-4_story_based_fake.json', 'r') as f:
    data = json.load(f)

# 创建新的数据结构并重新编号
formatted_data = []
new_id = 1

# for key, value in data.items():
for key, value in data.items():
    news_item = {
        "news_id": new_id,
        "title": value["origin_title"],
        "body_text": value["generated_text"],
        "reliability": 1 if value["origin_label"] == "real" else 0
    }
    formatted_data.append(news_item)
    new_id += 1

    # 限定最大ID为2029
    if new_id >= 2029:
        break

# 将新数据保存为JSON文件
with open('/content/drive/MyDrive/formatted_news.json', 'w') as f:
    json.dump(formatted_data, f)


In [69]:
import random
from sklearn.model_selection import train_test_split

# 读取格式化后的数据集
with open('/content/drive/MyDrive/formatted_news.json', 'r') as f:
    formatted_data = json.load(f)

# 打乱数据集
random.shuffle(formatted_data)

# 划分数据集
train_data, temp_data = train_test_split(formatted_data, test_size=0.2, random_state=42)
val_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)

# 保存划分后的数据集到我的Google drive
with open('/content/drive/MyDrive/train.json', 'w') as f:
    json.dump(train_data, f)

with open('/content/drive/MyDrive/val.json', 'w') as f:
    json.dump(val_data, f)

with open('/content/drive/MyDrive/test.json', 'w') as f:
    json.dump(test_data, f)


In [70]:
# 保存划分后的数据集
with open('/content/HERO/data/GossipCop/train.json', 'w') as f:
    json.dump(train_data, f)

with open('/content/HERO/data/GossipCop/val.json', 'w') as f:
    json.dump(val_data, f)

with open('/content/HERO/data/GossipCop/test.json', 'w') as f:
    json.dump(test_data, f)

In [ ]:
%run test.py

torch version : 2.3.0+cu121
cuda
===> loading  embed ...
embeding_example:  [-0.038194 -0.24487   0.72812  -0.39961   0.083172  0.043953 -0.39141
  0.3344   -0.57545   0.087459  0.28787  -0.06731   0.30906  -0.26384
 -0.13231  -0.20757   0.33395  -0.33848  -0.31743  -0.48336   0.1464
 -0.37304   0.34577   0.052041  0.44946  -0.46971   0.02628  -0.54155
 -0.15518  -0.14107  -0.039722  0.28277   0.14393   0.23464  -0.31021
  0.086173  0.20397   0.52624   0.17164  -0.082378 -0.71787  -0.41531
  0.20335  -0.12763   0.41367   0.55187   0.57908  -0.33477  -0.36559
 -0.54857  -0.062892  0.26584   0.30205   0.99775  -0.80481  -3.0243
  0.01254  -0.36942   2.2167    0.72201  -0.24978   0.92136   0.034514
  0.46745   1.1079   -0.19358  -0.074575  0.23353  -0.052062 -0.22044
  0.057162 -0.15806  -0.30798  -0.41625   0.37972   0.15006  -0.53212
 -0.2055   -1.2526    0.071624  0.70565   0.49744  -0.42063   0.26148
 -1.538    -0.30223  -0.073438 -0.28312   0.37104  -0.25217   0.016215
 -0.017099 -0.